In [3]:
from flask import Flask, request, jsonify, g as app_ctx
import pickle
import numpy as np
import torch
import cv2
import io
from matplotlib import pyplot as plt
import time
from position_calculator import calculate_position
from ultralytics import YOLO
#The Python Imaging Library adds image processing capabilities to your Python interpreter. 
import PIL
#Pickle is used for serializing and de-serializing Python object structures, 
#also called marshalling or flattening. Serialization refers to the process 
#of converting an object in memory to a byte stream that can be stored on 
#disk or sent over a network. Later on, this character stream can then be 
#retrieved and de-serialized back to a Python object. 
#Unpickling a file that was pickled in a different version of Python may 
#not always work properly
#https://www.datacamp.com/tutorial/pickle-python-tutorial

In [5]:
# This dictionary will be utilized while finding the distance of objects from the user. 
# We are using the width of an object instead of its height because while capturing the 
# frame, the complete height of the object might not be captured. The width is more likely
# to be appearing in full length.
objects_actual_width = {'person': 38.5, 'tv':5, 'bicycle': 175, 'couch': 152, 'bus': 1200, 'car': 448, 'chair': 46, 'motorcycle': 221, 
                       'traffic light': 40 , 'bed': 141, 'bench': 114, 'dining_table': 160, 'dog': 84, 'cat': 38}

In [6]:
navi_app = Flask(__name__)
# after the application starts, we can load the model for once 
# and make several requests to it
model = YOLO("yolov8n.pt") 

  0%|          | 0.00/6.23M [00:00<?, ?B/s]

In [7]:
# flask's before_request and after_request decorator to measure time taken for a request to complete.
@navi_app.before_request
def logging_before():
    # Store the start time for the request
    app_ctx.start_time = time.perf_counter()


@navi_app.after_request
def logging_after(response):
    # Get total time in milliseconds
    total_time = time.perf_counter() - app_ctx.start_time
    time_in_ms = int(total_time * 1000)
    # Log the time taken for the endpoint 
    print('Response time => ', time_in_ms, 'ms')
    return response

In [8]:
@navi_app.route('/predict', methods=["GET", "POST"])
def predict():
    if request.method == "GET":
        return "Please send post request"

    elif request.method == "POST":
        frame = request.files.get('frame') # get the frame sent by the API request 
        im_bytes = frame.read() # convert the file into byte stream
        image = PIL.Image.open(io.BytesIO(im_bytes)) # convert the byte stream into 
        image.show()
        image_width, image_height = image.size
        
        prediction = model(image)
        objects_with_positions = calculate_position(prediction.pandas().xyxy, objects_actual_width, image_width, image_height)
        
        data = {
            "objects_with_positions" : objects_with_positions,
        }
  
        return jsonify(data)

In [ ]:
navi_app.run(port=5000, host='0.0.0.0', debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.143.11.150:5000
Press CTRL+C to quit
10.143.11.150 - - [10/Apr/2023 15:28:23] "POST /predict HTTP/1.1" 200 -


Response time =>  20619 ms


10.143.11.150 - - [10/Apr/2023 15:28:44] "POST /predict HTTP/1.1" 200 -


Response time =>  19505 ms


...

Response time =>  19828 ms


10.143.11.150 - - [10/Apr/2023 16:28:19] "POST /predict HTTP/1.1" 200 -


Response time =>  19621 ms


[2023-04-10 16:28:40,145] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "C:\Users\hevra\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\hevra\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\hevra\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\hevra\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "C:\Users\hevra\AppData\Local\Temp\ipykernel_9092\3075169150.py", line 14, in predict
    objects_with_positions = calculate_position(prediction.pandas().xyxy, objects_actual_width, image_width, image_height)
  File "C:\Users\hevra\an

Response time =>  19788 ms
